In [2]:
#Get data for streets, driving, biking, walk paths
import osmnx as ox

#Library for analyziing complex networks and graphs
import networkx as nx

#Converts address to latitude and longitude
from geopy.geocoders import Nominatim

#Interactive Graphs
import folium

#Static Graphing
import matplotlib.pyplot as plt

#Display final map
from IPython.display import display

#Memory Efficent tools for working with iterations
import itertools

#Methods to get Geopy.geocoders to work
import ssl
import certifi

In [7]:
# Create an SSL context that uses certifi's certificate bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())

#Initialize Geolocater for the cordinates
geolocator = Nominatim(user_agent='wichita_nav', ssl_context=ssl_context)

# Generate road network for Wichita
G = ox.graph_from_place('Wichita, Kansas, USA', network_type='drive')

# Function to plot paths on the map
def plot_route(G, route, route_map, color='blue'):
    route_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_points, color=color, weight=4.5, opacity=0.7).add_to(route_map)

#Function to get nearest node from a location
def get_nearest_node(address): 
    location = geolocator.geocode(address, timeout=5)  # 5 sec timeout 
    if location:
        print(f"Geocoded Address: {address} -> ({location.latitude}, {location.longitude})")  # Debugging output
        return ox.distance.nearest_nodes(G, X=location.longitude, Y=location.latitude)
    else: 
        print(f"Failed to geocode address: {address}")  # Identify the issue
        return None

# Example start and end addresses
start_address = "7331 Ayesbury Cir, Wichita KS"
end_address = "123 Douglas Ave, Wichita, KS"

start_node = get_nearest_node(start_address)
end_node = get_nearest_node(end_address)

if start_node and end_node:
    #interactive map
    route_map = folium.Map(location=[37.6872, -97.3301], zoom_start=12)  # Centering on Wichita

    # After creating your graph G
    G_simple = nx.DiGraph(G)

    #Route 1 shortest path using Dijkstra's algorithm
    paths_generator = nx.shortest_simple_paths(G_simple, source=start_node, target=end_node, weight='length')
    
    #Create a similarity measure to find paths that differ from first
    def path_similarity(path1, path2):
        #Return ratio of overlaping nodes
        set1, set2 = set(path1), set(path2)

        #1 = paths are same, 0 = paths share no nodes
        return len(set1.intersection(set2)) / min(len(set1), len(set2))
    
    #Filter paths for differential
    def k_shortest_paths_differential (G, source, target, k=3, max_paths=100):
        accepted_paths = []
        paths_generator = nx.shortest_simple_paths(G, source, target, weight='length')

        #Iterate over possible paths
        for path in itertools.islice(paths_generator, max_paths):
            if not accepted_paths: 
                # Always accept the first (shortest) path.
                accepted_paths.append(path)
                print("Accepted first path with length:", nx.path_weight(G_simple, path, weight='length'))
            else:
                #Check similarity against other paths
                is_different = True
                for accepted in accepted_paths: 
                    similarity = path_similarity(path, accepted)
                    # Using (1 - threshold) as max allowable similarity
                    if similarity > .1: 
                        is_different = False
                        break
                if is_different: 
                    accepted_paths.append(path)
                    print("Accepted path with length:", nx.path_weight(G, path, weight='length'))
            if len(accepted_paths) >= k:
                break 
        
        if not accepted_paths:
            print("No sufficiently different routes found.")
        elif len(accepted_paths) == 1:
            print("Only one unique route was found.")

        return accepted_paths
    
    # Example: Get 3 sufficiently different routes.
    differential_routes = k_shortest_paths_differential(G_simple, start_node, end_node, k=3)


    # Define colors for different routes
    colors = ['blue', 'red', 'green', 'purple', 'orange']

    # Iterate through differential_routes and plot each one
    for idx, route in enumerate(differential_routes):
        #Cycle through colors
        plot_route(G, route, route_map, color=colors[idx % len(colors)])  


    folium.Marker([G.nodes[start_node]['y'], G.nodes[start_node]['x']], popup='Start').add_to(route_map)
    folium.Marker([G.nodes[end_node]['y'], G.nodes[end_node]['x']], popup='End').add_to(route_map)


     # Display map
    display(route_map)
else:
    print("Error: One of the addresses could not be geocoded.")


    

Geocoded Address: 7331 Ayesbury Cir, Wichita KS -> (37.727137306122444, -97.25127428571429)
Geocoded Address: 123 Douglas Ave, Wichita, KS -> (37.68591687755102, -97.33855526530613)
Accepted first path with length: 11579.928949104331
Only one unique route was found.
